# Vectorisation avec Python

## Opérations sur des vecteurs

Python fournit heureusement des outils pour effectuer les opérations sur les vecteurs. Parmi les librairies utiles, citons *math*, *Numpy*, *Scipy* et *Scikit-Learn*.

Avant de les charger en fonction de nos besoins, reprenons quelques vecteurs vus dans le précédent calepin :

$$
\vec{A} = \begin{pmatrix}
    3  \\
    12 \\
    9  \\
    0
\end{pmatrix}
\hspace{2em}
\vec{B} = \begin{pmatrix}
    7  \\
    32 \\
    10
\end{pmatrix}
\hspace{2em}
\vec{C} = \begin{pmatrix}
    11 \\
    4  \\
    8
\end{pmatrix}
$$

In [ ]:
A = [3, 12, 9, 0]
B = [7, 32, 10]
C = [11, 4, 8]

### Le produit scalaire de deux vecteurs

La méthode `.dot()` permet d’obtenir le produit de deux vecteurs :

In [ ]:
import numpy as np

np.dot(A, A)

### La norme d’un vecteur

Nous avions établi précédemment la norme du vecteur $\vec{A}$ approximativement à $15,2971$. Vérifions notre calcul avec la fonction d’algèbre linéaire `norm()` :

In [ ]:
np.linalg.norm(A)

### La distance entre deux vecteurs

La distance euclidienne entre deux vecteurs a cet avantage qu’elle peut se trouver dans un espace de n’importe quel dimension. L’opération revient à soustraire deux vecteurs puis à obtenir la norme de ce nouveau vecteur :

In [ ]:
np.linalg.norm(np.array(C) - np.array(B))

**Remarque :** notons que nous avons dû convertir explicitement nos variables `B` et `C`, de type `list` en tableaux *Numpy*. À titre anecdotique, le module `math` se passe de la conversion :

In [ ]:
from math import dist

dist(B, C)

## Métriques d’évaluation

### La similarité cosinus

Avec *Numpy*, il est nécessaire de déplier la formule :

In [ ]:
cos_BC = np.dot(B, C) / np.dot(np.linalg.norm(B), np.linalg.norm(C))

print(f"La similarité cosinus des vecteurs B et C est évaluée à {cos_BC * 100:.2f} %.")

Une fonction adéquate existe dans la librairie *Scikit-Learn*, `cosine_similarity`. Il convient toutefois de lui transmettre une matrice aux bonnes dimensions (nombre de vecteurs, nombre d’attributs). Dans notre exemple, la dimension serait (1, 3) parce que nous lui envoyons un vecteur constitué de trois composantes :

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cos_BC = cosine_similarity(
    np.array(B).reshape(1, -1),
    np.array(C).reshape(1, -1)
)

print(f"La similarité cosinus des vecteurs B et C est évaluée à {cos_BC.ravel().tolist()[0] * 100:.2f} %.")

### L’indice de Jaccard

Une fois encore, *Scikit-Learn* permet d’obtenir directement la métrique en comparant deux vecteurs :

In [ ]:
from sklearn.metrics import jaccard_score

A = [0, 0, 1, 1, 1]
B = [1, 0, 1, 0, 0]

print(f"Les vecteurs A et B sont similaires à {jaccard_score(A, B) * 100:.2f} %")

## Vectoriser un texte

Considérons un corpus restreint de trois phrases :

In [ ]:
corpus = [
    "Le petit chat boit du lait.",
    "Le petit chien boit de l’eau.",
    "La vache boit de l’eau mais ne boit pas de lait."
]

### L’approche fréquentielle

Le module *Scikit-Learn* dispose d’outils pour l’extraction de caractéristiques dans un texte :

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

Le résultat produit un vocabulaire, que l’on peut récupérer avec la méthode `.get_feature_names_out()` :

In [ ]:
vectorizer.get_feature_names_out()

La méthode `.toarray()` quant à elle permet de représenter la matrice creuse du corpus :

In [ ]:
X.toarray()

Associée à *Pandas*, il est possible de révéler le nom des colonnes afin de nous apprendre que dans la dernière phrase le mot *de* apparaît deux fois :

In [ ]:
import pandas as pd

df = pd.DataFrame(
    data=X.toarray(),
    columns=vectorizer.get_feature_names_out()
)

print(df)

Il est également possible de transmettre directement un vocabulaire au constructeur plutôt que de lui laisser la charge de le construire :

In [ ]:
vocabulary = ['boit', 'chat', 'chien', 'eau', 'lait', 'petit', 'vache']
vectorizer = CountVectorizer(vocabulary=vocabulary)
X = vectorizer.fit_transform(corpus)

X.toarray()

### L’encodage *one-hot*

Si l’on souhaite n’obtenir qu’un encodage *one-hot*, sans le décompte des occurrences du mot dans la phrase, il suffit de transformer la matrice de sortie :

In [ ]:
list(map(lambda row: [ 0 if e == 0 else 1 for e in row ], X.toarray()))

### La méthode TF-IDF

À partir d’une matrice d’occurrences, obtenue par exemple avec le transformateur `CountVectorizer`, il est possible de calculer une matrice TF (*term frequency*) ou TF-IDF (*term frequency-inverse document frequency*). C’est le rôle du transformateur `TfidfTransformer` :

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
result = tfidf.fit_transform(X)

Le résultat est une matrice creuse qu’il est possible d’afficher tel quel :

In [ ]:
print(result)

Chaque ligne donne la mesure TF-IDF d’un mot dans un document. Prenons la ligne :

```txt
(1, 5)	0.4804583972923858
```

Il faut comprendre ici que dans le document n°1, la mesure TF-IDF du terme n°5 est de 0,48046. Autrement dit, l’importance du terme *petit* est évaluée à 0,48046 dans la deuxième phrase de notre corpus.

Il est à noter que le transformateur ne garde pas trace des caractéristiques apprises, juste de leurs indices :

In [ ]:
tfidf.get_feature_names_out()

Pour les retrouver, il faut interroger la matrice d’occurrences :

In [ ]:
vectorizer.get_feature_names_out()